#### Урок 3. Парсинг HTML. BS, SQLAlchemy

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД
2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы
3*)Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [146]:
# Установка БД
"""
docker run -d -p 127.0.0.1:27017:27017 --name mongo4parsing mongo
docker ps -a
docker start f48a0957b452
docker exec -it f48a0957b452 mongo

"""

'\ndocker run -d -p 127.0.0.1:27017:27017 --name mongo4parsing mongo\ndocker ps -a\ndocker start f48a0957b452\ndocker exec -it f48a0957b452 mongo\n\n'

In [151]:
from pymongo import MongoClient
#from pprint import pprint
import pandas as pd
import numpy as np
from pprint import pprint

client = MongoClient('mongodb://127.0.0.1:27017')
vacancy_pd = pd.read_pickle('vacancy_pd.pkl')

In [152]:
vacancy_pd
type(vacancy_pd["min_salary"][3])

list

In [153]:
def db_vacancy_writer(df):
    client.vacancy.docs.drop()
    vacancy_db = client['vacancy']
    docs = vacancy_db.docs
    result = docs.insert_many(df.to_dict(orient='records'))
    return result.inserted_ids

In [154]:
client.vacancy.docs.drop()
db_vacancy_writer(vacancy_pd)


[ObjectId('5db42f16aba5db6b7932ae5e'),
 ObjectId('5db42f16aba5db6b7932ae5f'),
 ObjectId('5db42f16aba5db6b7932ae60'),
 ObjectId('5db42f16aba5db6b7932ae61'),
 ObjectId('5db42f16aba5db6b7932ae62'),
 ObjectId('5db42f16aba5db6b7932ae63'),
 ObjectId('5db42f16aba5db6b7932ae64'),
 ObjectId('5db42f16aba5db6b7932ae65'),
 ObjectId('5db42f16aba5db6b7932ae66'),
 ObjectId('5db42f16aba5db6b7932ae67'),
 ObjectId('5db42f16aba5db6b7932ae68'),
 ObjectId('5db42f16aba5db6b7932ae69'),
 ObjectId('5db42f16aba5db6b7932ae6a'),
 ObjectId('5db42f16aba5db6b7932ae6b'),
 ObjectId('5db42f16aba5db6b7932ae6c'),
 ObjectId('5db42f16aba5db6b7932ae6d'),
 ObjectId('5db42f16aba5db6b7932ae6e'),
 ObjectId('5db42f16aba5db6b7932ae6f'),
 ObjectId('5db42f16aba5db6b7932ae70'),
 ObjectId('5db42f16aba5db6b7932ae71'),
 ObjectId('5db42f16aba5db6b7932ae72'),
 ObjectId('5db42f16aba5db6b7932ae73'),
 ObjectId('5db42f16aba5db6b7932ae74'),
 ObjectId('5db42f16aba5db6b7932ae75'),
 ObjectId('5db42f16aba5db6b7932ae76'),
 ObjectId('5db42f16aba5db

In [155]:
objects = docs.find()
pprint(objects)
for obj in objects:
    
    pprint(obj)

{'_id': ObjectId('5db42f16aba5db6b7932ae5e'),
 'employer': [' Всероссийский Банк Развития Регионов (ВБРР)'],
 'link': ['https://hh.ru'],
 'max_salary': [nan],
 'min_salary': [nan],
 'vacancy': ['Методолог кадрового администрирования'],
 'vacancy_link': ['https://hh.ru/vacancy/34240183?query=%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D0%BE%D0%BB%D0%BE%D0%B3']}
{'_id': ObjectId('5db42f16aba5db6b7932ae5f'),
 'employer': [' Ренессанс cтрахование, Группа'],
 'link': ['https://hh.ru'],
 'max_salary': [nan],
 'min_salary': [nan],
 'vacancy': ['Методолог/Руководитель направления методологии в Управление '
             'бухгалтерского учета'],
 'vacancy_link': ['https://hh.ru/vacancy/34005950?query=%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D0%BE%D0%BB%D0%BE%D0%B3']}
{'_id': ObjectId('5db42f16aba5db6b7932ae60'),
 'employer': [' Ренессанс cтрахование, Группа'],
 'link': ['https://hh.ru'],
 'max_salary': [nan],
 'min_salary': [nan],
 'vacancy': ['Ведущий специалист/Риск-менеджер в Управление риск-менеджмента'],
 'vacanc

 'employer': ['Стаффмикс'],
 'link': ['https://www.superjob.ru'],
 'max_salary': [nan],
 'min_salary': [nan],
 'vacancy': ['Системный аналитик'],
 'vacancy_link': ['https://www.superjob.ru/vakansii/sistemnyj-analitik-32771986.html']}
{'_id': ObjectId('5db42f16aba5db6b7932aec9'),
 'employer': ['ФГБУ ЦКБ с поликлиникой УД Президента РФ'],
 'link': ['https://www.superjob.ru'],
 'max_salary': [nan],
 'min_salary': [80000],
 'vacancy': ['Врач-методист'],
 'vacancy_link': ['https://www.superjob.ru/vakansii/vrach-metodist-32337051.html']}
{'_id': ObjectId('5db42f16aba5db6b7932aeca'),
 'employer': ['Компания "Ронова"'],
 'link': ['https://www.superjob.ru'],
 'max_salary': [nan],
 'min_salary': [nan],
 'vacancy': ['Бизнес-аналитик / Ведущий аналитик 1С'],
 'vacancy_link': ['https://www.superjob.ru/vakansii/biznes-analitik-32625772.html']}
{'_id': ObjectId('5db42f16aba5db6b7932aecb'),
 'employer': ['СОГАЗ'],
 'link': ['https://www.superjob.ru'],
 'max_salary': [nan],
 'min_salary': [nan],
 'vaca

In [303]:
def db_vacancy_search(salary):
    vacancy_db = client['vacancy']
    docs = vacancy_db.docs
    collections = docs.find({ "min_salary": {'$gte':salary} }).sort("min_salary")
    for x in collections:
        print(*x['vacancy'], ": зарплата от ", *x['min_salary'], " до ",*x['max_salary'])
        
    a = docs.find({ "max_salary": {'$gte':salary} } ).sort("max_salary")# 
    for x in a:
        print(*x['vacancy'], ": зарплата до ",*x['max_salary'])



In [304]:
salary = 100000
db_vacancy_search(salary)

Системный аналитик : зарплата от  100000  до  150000
Программист 1C : зарплата от  100000  до  200000
Старший маркетолог-аналитик : зарплата от  105000  до  nan
Ведущий специалист по внутреннему аудиту : зарплата от  110000  до  nan
Начальник отдела финансового мониторинга : зарплата от  110000  до  nan
Врач-методист : зарплата от  110000  до  nan
Менеджер проектного офиса : зарплата от  115000  до  nan
Методолог-консультант : зарплата от  120000  до  120000
Методолог : зарплата от  120000  до  nan
Руководитель проектного офиса : зарплата от  120000  до  160000
Системный аналитик : зарплата от  130000  до  160000
Системный аналитик (чат-боты) : зарплата от  149500  до  195000
Начальник отдела проектного офиса (IT-компания) : зарплата от  150000  до  200000
Бизнес-аналитик : зарплата от  150000  до  nan
Заместитель главного бухгалтера по работе с дочерними обществами : зарплата от  180000  до  nan
Методолог-консультант : зарплата до  100000
Менеджер проектов : зарплата до  100000
Внутре

In [313]:
# Пока не получилость сделать проверку если не указана минимальная ЗП, а максимальная больше-равно заданной

salary = 100000
vacancy_db = client['vacancy']
docs = vacancy_db.docs

a = docs.find({ "min_salary": {}})

#a = docs.find({ "max_salary": {'$gte':salary} and "min_salary": {'nan'}}).sort("max_salary") 
print(a)
for x in a:
#    print(x)
    print(*x['vacancy'], ": зарплата от ", *x['min_salary'], " до ",*x['max_salary'])




In [ ]:
### пример

import pandas as pd
from pymongo import MongoClient


def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df